### Configuração da conexão do Databricks ao armazenamento Azure

Código para conexão entre o databricks e o armazenamento na Azure. A conclusão dessa operação exige que o aplicativo criado para o Databricks tenha acesso à conta de armazenamento.

Guia para configurar o controle de acesso:

1. Acessar conta de armazenamento do datalake no portal da Azure
2. Selecionar opção 'IAM (Controle de Acesso)'
3. Procurar e selecionar função 'Colaborador de Dados do Blob Storage'
4. Selecionar a opção 'Selecionar Membros'
5. Procurar e selecionar o aplicativo criado para conexão

In [ ]:
# Variáveis para as credenciais de acesso

id_cliente = 'inserir_seu_id'
id_locatario = 'inserir_seu_locatario'
senha_app = 'inserir_sua_senha'  # Precisa ser gerada
nome_conta_de_armazenamento = 'datalakexp'  # Nome da conta de armazenamento do datalake

### Inicializando a sessão Spark

Essa etapa não é necessária quando se está no ambiente em nuvem do databricks.

In [ ]:
# Iniciando a sessão spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkSession").getOrCreate()

In [ ]:
spark.conf.set(f"fs.azure.account.auth.type.{nome_conta_de_armazenamento}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{nome_conta_de_armazenamento}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{nome_conta_de_armazenamento}.dfs.core.windows.net", id_cliente)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{nome_conta_de_armazenamento}.dfs.core.windows.net", senha_app)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{nome_conta_de_armazenamento}.dfs.core.windows.net", f"https://login.microsoftonline.com/{id_locatario}/oauth2/token")

### Definição dos MOUNT's

Um MOUNT é um ponto de acesso, usado para facilitar o acesso a pastas.

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": id_cliente,
          "fs.azure.account.oauth2.client.secret": senha_app,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{id_locatario}/oauth2/token"}

# Ponto de acesso da camada BRONZE
dbutils.fs.mount(
  source = f"abfss://bronze@{nome_conta_de_armazenamento}.dfs.core.windows.net/",
  mount_point = "/mnt/datalake/bronze",
  extra_configs = configs
)

# Ponto de acesso da camada SILVER
dbutils.fs.mount(
  source = f"abfss://silver@{nome_conta_de_armazenamento}.dfs.core.windows.net/",
  mount_point = "/mnt/datalake/silver",
  extra_configs = configs
)

# Ponto de acesso da camada GOLD
dbutils.fs.mount(
  source = f"abfss://gold@{nome_conta_de_armazenamento}.dfs.core.windows.net/",
  mount_point = "/mnt/datalake/gold",
  extra_configs = configs
)

### Checando os mounts criados

display(dbutils.fs.mounts())